# Texi Guru Kaggle Challange
* Step 1: Data importing

In [26]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


##### Reading test data and train data

In [61]:
train_pd = pd.read_csv('train.csv')
copy_train_pd = train_pd.copy()
test_pd = pd.read_csv('test.csv')
copy_test_pd = test_pd.copy()
train_pd.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               175000 non-null  int64  
 1   tpep_pickup_datetime   175000 non-null  object 
 2   tpep_dropoff_datetime  175000 non-null  object 
 3   passenger_count        168923 non-null  float64
 4   trip_distance          175000 non-null  float64
 5   RatecodeID             168923 non-null  float64
 6   store_and_fwd_flag     168923 non-null  object 
 7   PULocationID           175000 non-null  int64  
 8   DOLocationID           175000 non-null  int64  
 9   payment_type           175000 non-null  object 
 10  extra                  175000 non-null  float64
 11  tip_amount             175000 non-null  float64
 12  tolls_amount           175000 non-null  float64
 13  improvement_surcharge  175000 non-null  float64
 14  total_amount           175000 non-nu

##### Computing the null values and missing values from the tables

In [62]:
train_pd

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.00
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.00
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.00
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.00
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174995,1,2023-06-30 22:50:57,2023-06-30 22:22:22,3.0,3.45,1.0,N,147,167,Credit Card,1.0,8.732495,0.0,1.0,28.08,2.5,0.00
174996,1,2023-06-30 13:03:33,2023-06-30 14:04:57,1.0,9.44,1.0,N,154,191,Cash,5.0,0.283275,0.0,1.0,59.95,2.5,1.75
174997,0,2023-06-29 11:03:32,2023-06-29 12:13:34,1.0,2.40,1.0,N,168,106,Credit Card,2.5,4.245354,0.0,1.0,33.50,2.5,0.00
174998,1,2023-06-29 19:47:17,2023-06-29 19:08:55,1.0,4.71,1.0,N,240,100,Credit Card,2.5,10.479776,0.0,1.0,40.80,2.5,0.00


In [63]:
train_pd.isna().sum()

VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          6077
trip_distance               0
RatecodeID               6077
store_and_fwd_flag       6077
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
congestion_surcharge     6077
Airport_fee              6077
dtype: int64

In [64]:
train_pd = copy_train_pd.copy()

y = train_pd['total_amount']
X = train_pd.drop('total_amount', axis=1) # seprating prediction element from training set
list_of_features = list(train_pd)


In [48]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [49]:
x_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee
123929,0,2023-06-29 09:25:12,2023-06-29 08:23:14,1.0,1.50,1.0,N,217,214,Credit Card,2.5,5.637803,0.00,1.0,2.5,0.00
5511,0,2023-06-28 18:43:46,2023-06-28 19:14:50,2.0,1.10,1.0,N,143,202,Credit Card,5.0,3.623210,0.00,1.0,2.5,0.00
9423,1,2023-06-30 15:23:10,2023-06-30 15:42:40,2.0,1.41,1.0,N,100,237,Credit Card,0.0,5.322425,0.00,1.0,2.5,0.00
117492,1,2023-06-29 08:42:23,2023-06-29 10:03:44,2.0,17.94,2.0,N,114,170,Credit Card,0.0,19.759234,6.55,1.0,2.5,1.75
48071,1,2023-06-30 01:41:44,2023-06-30 01:16:08,2.0,1.81,1.0,N,196,241,Credit Card,1.0,5.547569,0.00,1.0,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151494,1,2023-06-29 12:31:49,2023-06-29 12:55:39,4.0,8.62,1.0,N,97,75,Credit Card,5.0,0.620737,6.55,1.0,2.5,1.75
37653,0,2023-06-29 21:28:17,2023-06-29 20:55:36,1.0,1.70,1.0,N,248,107,Credit Card,3.5,3.499244,0.00,1.0,2.5,0.00
101564,1,2023-06-29 18:41:16,2023-06-29 20:13:17,1.0,2.93,1.0,N,221,236,Credit Card,2.5,7.664683,0.00,1.0,2.5,0.00
59017,1,2023-06-29 21:27:57,2023-06-29 22:06:45,2.0,0.58,1.0,N,181,63,Credit Card,1.0,5.147958,0.00,1.0,2.5,0.00


## Preprocessing section
* selecting the features for imputation
* transform each feature into same scale
* plotting each feature with respect to each one
* finding the corelation between features

In [50]:
print('Null values in the given training data set is:')
x_train.isna().sum()

Null values in the given training data set is:


VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          4817
trip_distance               0
RatecodeID               4817
store_and_fwd_flag       4817
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
congestion_surcharge     4817
Airport_fee              4817
dtype: int64

In [51]:
print(train_pd['RatecodeID'].median())
train_pd['Airport_fee'].unique()

1.0


array([ 0.  ,  1.75,   nan, -1.75])

So, in current transformer we are assuing if there is no fwd flag available then we data is not forwarded
Similary in case of congestion surcharge and Airport fees we are replacing ever NAN value with 0

In [56]:
def apply_transformations(data:pd.DataFrame) -> pd.DataFrame:
  pipe_for_fwd_imputr = Pipeline([
    ('simple_imputation', SimpleImputer(strategy='constant', fill_value='null', missing_values=np.nan)),
    ('fwd_one_hot_encoder', OneHotEncoder())
    ])
  pipe_for_payment_methods = Pipeline(
    [
      ('payment_one_hot_encoder', OneHotEncoder())
    ]
  )
  column_transformers = ColumnTransformer([
    ('pass0', 'passthrough', [0]),
    # ('pass1', 'passthrough', [1]),
    # ('pass2', 'passthrough', [2]),
    ('passanger_count_imputer', SimpleImputer(strategy='constant', fill_value=1, missing_values=np.nan), [3]),
    ('pass4', 'passthrough', [4]),
    ('rate_code_id_imputer', SimpleImputer(strategy='constant', fill_value=1.0, missing_values=np.nan), [5]),
    # ('store_and_fwd_flag_imputer',pipe_for_fwd_imputr, [6]),
    ('pass_', 'passthrough', [6]),
    ('pass6', 'passthrough', [7]),
    ('pass8', 'passthrough', [8]),
    ('payment_type_one_hot_encoding', pipe_for_payment_methods, [9]),
    ('pass10', 'passthrough', [10]),
    ('pass11', 'passthrough', [11]),
    ('pass12', 'passthrough', [12]),
    ('pass13', 'passthrough', [13]),
    ('congestion_surcharge_imputer', SimpleImputer(strategy='constant', fill_value=0, missing_values=np.nan), [14]),
    ('airport_fee_imputer', SimpleImputer(strategy='constant', fill_value=-5, missing_values=np.nan), [15])
  ])
  # column_pass_through = ColumnTransformer([
    
    
  
  # ])

  feature_pipeline = Pipeline([
    ('column_transformers', column_transformers)
    # ('columns_pass_by', column_pass_through)
  ])
  val = feature_pipeline.fit_transform(data)
  return pd.DataFrame(data=val)
  # return pd.DataFrame()

In [60]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,1.0,1.50,1.0,1.0,0.0,0.0,217.0,214.0,0.0,1.0,0.0,0.0,0.0,2.5,5.637803,0.00,1.0,2.5,0.00
1,0.0,2.0,1.10,1.0,1.0,0.0,0.0,143.0,202.0,0.0,1.0,0.0,0.0,0.0,5.0,3.623210,0.00,1.0,2.5,0.00
2,1.0,2.0,1.41,1.0,1.0,0.0,0.0,100.0,237.0,0.0,1.0,0.0,0.0,0.0,0.0,5.322425,0.00,1.0,2.5,0.00
3,1.0,2.0,17.94,2.0,1.0,0.0,0.0,114.0,170.0,0.0,1.0,0.0,0.0,0.0,0.0,19.759234,6.55,1.0,2.5,1.75
4,1.0,2.0,1.81,1.0,1.0,0.0,0.0,196.0,241.0,0.0,1.0,0.0,0.0,0.0,1.0,5.547569,0.00,1.0,2.5,0.00


In [65]:

# # df = apply_transformations(train_pd)
# x_train['tpep_pickup_datetime'] = pd.to_datetime(x_train['tpep_pickup_datetime'])
# x_train['tpep_dropoff_datetime'] = pd.to_datetime(x_train['tpep_dropoff_datetime'])

# x_test['tpep_pickup_datetime'] = pd.to_datetime(x_test['tpep_pickup_datetime'])
# x_test['tpep_dropoff_datetime'] = pd.to_datetime(x_test['tpep_dropoff_datetime'])
# x_train = apply_transformations(x_train)
# x_test = apply_transformations(x_train)
 #! generating errors need to check 


In [66]:
x_train_non_number_f = x_train.select_dtypes(exclude="number")
x_train_non_number_f.head()
x_train_copy = x_train.copy()

In [68]:
# x_train = x_train_copy.copy()
# x_train = x_train.drop(['tpep_dropoff_datetime', 'tpep_pickup_datetime'], axis=1)
# x_train = x_train.drop(['store_and_fwd_flag', 'payment_type'], axis=1)
# x_train.info()


### Checking null values

In [69]:
x_train.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
dtype: int64

### dummy model for milestone 1

In [70]:
from sklearn.dummy import DummyRegressor

In [75]:
dummy_regressor = DummyRegressor(strategy= 'mean')

In [76]:
dummy_regressor.fit(x_train, y_train)

DummyRegressor()

In [78]:
dummy_regressor.score(x_train, y_train)

0.0

In [79]:
dummy_regressor.score(x_test, y_test)

-2.6232709090745487e-05

In [83]:
dummy_output = dummy_regressor.predict(x_test)
dummy_output = pd.DataFrame(dummy_output)

In [84]:
dummy_output.to_csv('submission.csv')